<h1> The linear span of uniform matrix product states </h1>

We consider the variety of uniform matrix product states $\operatorname{uMPS}(2,2,d) \subset \operatorname{Dih}^d(\mathbb{C}^2)$. Its ambient space $V=\operatorname{Dih}^d(\mathbb{C}^2)$ is a $GL_2$-representation. 
Our code computes the degree $k$ part $I_k$ of the ideal $I$ defining $\operatorname{uMPS}(2,2,d)$, which is a subspace of the $GL_2$-representation $S^kV^*$.

More precisely, for every $d$ from $4$ to $\verb|max_d|$, we compute three lists:
<ul>
    <li>$\verb|ambientCharacter|$ is the character of the representation $S^kV$ (if $k=1$, this is the ambient space).</li>
    <li>$\verb|idealCharacter|$ is the character of the representation $I_k$.</li>
    <li>$\verb|quotientCharacter|$ is the character of the quotient $S^kV$ (if $k=1$, this is the linear span $\langle \operatorname{uMPS}(2,2,d) \rangle$).</li>
</ul>


In [1]:
K=QQ
import itertools

#Function polynomialSpan
#Input: polynomialList: a list of polynomials
#Output: the dimension of their linear span
def polynomialSpan(polynomialList):
    monomials = set(flatten([P.monomials() for P in polynomialList]))
    M=matrix([[P.monomial_coefficient(mon) for P in polynomialList] for mon in monomials])
    return rank(M)

#Function wordToBracelet
#Input: w: a word in the alphabet {0,1}
#Output: the corresponding bracelet, which is represented by the lexicographically largest word
#that can be obtained from w under the action of the dihedral group
#This function is only used when calling makeWordsAndBracelets below.
def wordToBracelet(w):
    w=list(w)
    b = w[:]
    for i in range(len(w)-1):
        w = w[1:]+[w[0]]
        if b < w:
            b=w[:]
    w.reverse()
    for i in range(len(w)):
        w = w[1:]+[w[0]]
        if b < w:
            b=w[:]
    return(tuple(b))

#Instead of repeatedly running the function wordToBracelet over the course of our algorithm,
#we run it once for each word and store the result in a dictionary:
wordToBraceletDict = {}

#Function makeWordsAndBracelets
#Input: a natural number N
#Output: a pair (list all words of length N, list all bracelets of length N)
#Moreover, it adds the pairs (word, bracelet) to the aforementioned dictionary 
def makeWordsAndBracelets(N):
    words = IntegerVectors(length=N,max_part=1)
    bracelets = []
    for w in words:
        b=wordToBracelet(w)
        if b not in bracelets:
            bracelets.append(b)
        wordToBraceletDict[tuple(w)]=b
    return (words,bracelets)


#Initialize wordToBraceletDict
makeWordsAndBracelets(1);
makeWordsAndBracelets(2);

#The dictionary H has as keys words in {0,1},
#and as values the corresponding monomials in the trace parametrization of uMPS
#We implement this by hand for d<4 
S= PolynomialRing(K,5,"T")
T=S.gens()
H = {}
H[(0,)] = T[0]
H[(1,)] = T[1]
H[(0,0)]=T[2]
H[(1,0)]=T[3]
H[(0,1)]=T[3]
H[(1,1)]=T[4]
(words,bracelets)=makeWordsAndBracelets(3)
for b in bracelets:
    b=tuple(b)
    H[b]=(H[(b[0],)]*(H[(b[1],b[2])])+H[(b[1],)]*(H[(b[2],b[0])])+H[(b[2],)]*(H[(b[0],b[1])])-H[(b[0],)]*H[(b[1],)]*H[(b[2],)])/2
for w in words:
    H[tuple(w)]=H[wordToBraceletDict[tuple(w)]]

#A global variable that keeps track of how far we computed the trace parametrization so far:
max_d=3

In [2]:
def computeCharacter(d,k):
    if d> max_d:
        for e in [max_d+1..d]:
            #We compute the trace parametrization of uMPS(2,2,d) and store it in the dict H 
            (words,bracelets)=makeWordsAndBracelets(e)
            for b in bracelets:
                b=tuple(b)
                A=(b[0],)
                B=(b[1],)
                C=(b[2],)
                D=b[3:]
                H[b]=( H[A]*(H[B+C+D]-H[B]*H[C+D])
                      +H[B]*(H[C+D+A]-H[C]*H[D+A])
                      +H[C]*(H[D+A+B]-H[D]*H[A+B])
                      +H[D]*(H[A+B+C]-H[A]*H[B+C])
                      -H[A+C]*H[B+D]
                      +H[A+B]*H[C+D]
                      +H[A+D]*H[B+C]
                      +H[A]*H[B]*H[C]*H[D] )/2
            for w in words:
                H[tuple(w)]=H[wordToBraceletDict[tuple(w)]]
    
    #monomials sorted by weight
    monsByWeight = [[] for i in range(k*d+1)]
    for bTuple in itertools.combinations_with_replacement(bracelets,k):
        w=sum([sum(b) for b in bTuple])
        monsByWeight[w]=monsByWeight[w]+[bTuple]
    
    ambientCharacter=[len(x) for x in monsByWeight]
    quotientCharacter=[]
    for w in range(k*d+1):
        quotientCharacter.append(polynomialSpan([prod([H[b] for b in p]) for p in monsByWeight[w]]))
    idealCharacter=[ambientCharacter[i]-quotientCharacter[i] for i in range(k*d+1)]
    return((ambientCharacter,idealCharacter,quotientCharacter))

In [3]:
#To obtain table 1
for d in [4..15]:
    print(str(d)+': '+str(computeCharacter(d,1)[2]))

4: [1, 1, 2, 1, 1]
5: [1, 1, 2, 2, 1, 1]
6: [1, 1, 3, 3, 3, 1, 1]
7: [1, 1, 3, 4, 4, 3, 1, 1]
8: [1, 1, 4, 5, 7, 5, 4, 1, 1]
9: [1, 1, 4, 6, 8, 8, 6, 4, 1, 1]
10: [1, 1, 5, 7, 11, 11, 11, 7, 5, 1, 1]
11: [1, 1, 5, 8, 12, 14, 14, 12, 8, 5, 1, 1]
12: [1, 1, 6, 9, 15, 17, 20, 17, 15, 9, 6, 1, 1]
13: [1, 1, 6, 10, 16, 20, 23, 23, 20, 16, 10, 6, 1, 1]
14: [1, 1, 7, 11, 19, 23, 29, 29, 29, 23, 19, 11, 7, 1, 1]
15: [1, 1, 7, 12, 20, 26, 32, 35, 35, 32, 26, 20, 12, 7, 1, 1]


In [4]:
#To obtain table 2
for d in [4..10]:
    print(str(d)+': '+str(computeCharacter(d,2)[1]))

4: [0, 0, 0, 0, 0, 0, 0, 0, 0]
5: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
6: [0, 0, 0, 0, 1, 1, 2, 1, 1, 0, 0, 0, 0]
7: [0, 0, 0, 0, 1, 3, 6, 7, 6, 3, 1, 0, 0, 0, 0]
8: [0, 0, 0, 0, 5, 10, 25, 32, 42, 32, 25, 10, 5, 0, 0, 0, 0]
9: [0, 0, 0, 1, 7, 21, 48, 79, 110, 119, 110, 79, 48, 21, 7, 1, 0, 0, 0]
10: [0, 0, 0, 1, 14, 38, 100, 176, 290, 360, 408, 360, 290, 176, 100, 38, 14, 1, 0, 0, 0]


In [5]:
#To obtain table 3
for d in [4..9]:
    print(str(d)+': '+str(computeCharacter(d,3)[1]))

4: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
5: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
6: [0, 0, 0, 0, 1, 2, 8, 11, 17, 17, 17, 11, 8, 2, 1, 0, 0, 0, 0]
7: [0, 0, 0, 0, 1, 4, 15, 29, 49, 67, 77, 77, 67, 49, 29, 15, 4, 1, 0, 0, 0, 0]
8: [0, 0, 0, 0, 5, 14, 51, 101, 198, 292, 414, 478, 532, 478, 414, 292, 198, 101, 51, 14, 5, 0, 0, 0, 0]
9: [0, 0, 0, 1, 7, 26, 83, 191, 388, 671, 1039, 1431, 1784, 1983, 1983, 1784, 1431, 1039, 671, 388, 191, 83, 26, 7, 1, 0, 0, 0]
